<a href="https://colab.research.google.com/github/bathl01/deep_learning_challenge_21/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Found this in https://keras.io/keras_tuner/ when looking for how to tune
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.1 MB/s eta 0:00:00


## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow import keras
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

#  Import and read the charity_data.csv.
import pandas as pd
charity_data_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
display(charity_data_df.head())
display(charity_data_df.tail())

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1
34298,996086871,WATERHOUSE CHARITABLE TR,T3,Independent,C1000,Preservation,Co-operative,1,1M-5M,N,36500179,0


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
charity_data_df = charity_data_df.drop(columns=["EIN"])
# charity_data_df = charity_data_df.drop(columns=["EIN", "NAME"])
charity_data_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Drop the additional ID columns, 'AFFILIATION' and 'ORGANIZATION'.
# charity_data_df = charity_data_df.drop(columns=["USE_CASE"])
# charity_data_df = charity_data_df.drop(columns=["AFFILIATION", "ORGANIZATION", "USE_CASE"])
# charity_data_df

In [5]:
# Determine the number of unique values in each column.
charity_data_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [6]:
# Determine the data type of each column.
charity_data_df.dtypes

NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [7]:
# Look at APPLICATION_TYPE value counts for binning
type_cnt = charity_data_df['APPLICATION_TYPE'].value_counts()
type_cnt

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
replace_list = list(type_cnt.iloc[8:].index)
replace_list

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [9]:
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_cnt.iloc[5:].index)
cutoff_value_type = 500
# Replace in dataframe
for app in application_types_to_replace:
    charity_data_df['APPLICATION_TYPE'] = charity_data_df['APPLICATION_TYPE'].replace(app,"Other_type")

# Check to make sure binning was successful
charity_data_df['APPLICATION_TYPE'].value_counts()

T3            27037
Other_type     2266
T4             1542
T6             1216
T5             1173
T19            1065
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
class_values = charity_data_df['CLASSIFICATION'].value_counts()
class_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_cnts_gt1 = class_values.loc[class_values > 1]
class_cnts_gt1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
replace_class = list(class_values.iloc[5:].index)
replace_class
class_values.iloc[5:].index

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object')

In [13]:
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_values.iloc[5:].index)
cutoff_value_class = 1000

# Replace in dataframe
for cls in classifications_to_replace:
    charity_data_df['CLASSIFICATION'] = charity_data_df['CLASSIFICATION'].replace(cls,"Other_class")

# Check to make sure binning was successful
charity_data_df['CLASSIFICATION'].value_counts()

C1000          17326
C2000           6074
C1200           4837
Other_class     2261
C3000           1918
C2100           1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Look at Name value counts for binning
name_cnt = charity_data_df['NAME'].value_counts()
name_cnt_gt = name_cnt.loc[name_cnt > 100]
name_cnt_gt

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [15]:
# Choose a cutoff value and create a list of names to be replaced
replace_name = list(class_values.iloc[30:].index)
replace_name
name_cnt.iloc[30:].index

Index(['AIR FORCE ASSOCIATION', 'WORKERS UNITED', 'GAMMA THETA UPSILON',
       'INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS',
       'SOCIETY OF SAINT VINCENT DE PAUL COUNCIL OF LOS ANGELES',
       'ROTARY INTERNATIONAL', 'CALIFORNIA GARDEN CLUBS INC',
       'SWEET ADELINES INTERNATIONAL', 'DISABLED AMERICAN VETERANS',
       'KLEIN SUPPORT GROUPS INC',
       ...
       'PSSST', 'EASTERN OREGON WORKFORCE INVESTMENT BOARD',
       'OUR PERFECT STORM', 'RED SHOE PROJECT INC',
       'CHRISTIAN DEAF FELLOWSHIP', 'ST LOUIS SLAM WOMENS FOOTBALL',
       'AIESEC ALUMNI IBEROAMERICA CORP', 'WEALLBLEEDRED ORG INC',
       'AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI',
       'WATERHOUSE CHARITABLE TR'],
      dtype='object', length=19538)

In [16]:
# use the variable name `names_to_replace`
names_to_replace = list(name_cnt.iloc[30:].index)
cutoff_value_class = 100

# Replace in dataframe
for cls in names_to_replace:
    charity_data_df['NAME'] = charity_data_df['NAME'].replace(cls,"LT100_Occurance")

# Check to make sure binning was successful
charity_data_df['NAME'].value_counts()

LT100_Occurance                                                       25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [17]:
# identify column types
charity_data_df.dtypes

NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_data_num_df = pd.get_dummies(charity_data_df)
display(cat_data_num_df.head())
display(cat_data_num_df.tail())

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34298,1,36500179,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [19]:
# Split our preprocessed data into our features and target arrays
y = cat_data_num_df["IS_SUCCESSFUL"].values
x = cat_data_num_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10, stratify=y)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [21]:
x_train_scaled

array([[ 0.01247081, -0.02866118, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01247081, -0.02866118, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01247081, -0.02866118, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687],
       ...,
       [ 0.01247081, -0.02866118, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01247081, -0.02743269, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01247081, -0.02866118, -0.09393549, ..., -0.07529085,
         0.02925687, -0.02925687]])

## Compile, Train and Evaluate the Model
### Attempt 3 - switched to auto-optimization using keras-tuner to get > 75% accuracy

In [22]:
# Write a function that creates and returns a Keras model.
def build_model(hp):
    nn_opt = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])


    nn_opt.add(tf.keras.layers.Dense(
        # tune the number of units
        units=hp.Int("units", min_value=1, max_value=5, step=2),
            activation=activation, input_dim=x_train_scaled.shape[1]))

     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):    # can go to 5
        nn_opt.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=5,   # play with this from 5
            step=2),
            activation=activation))

    nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # compile the model
    nn_opt.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_opt


In [23]:
 # Initialize a tuner (here, RandomSearch). We use objective to specify the objective to select the best models, and we use max_trials to specify the number of different models to try.
 tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

 tuner.search(x=x_train_scaled, y=y_train, epochs=20, validation_data=(x_test_scaled, y_test), batch_size=32)

Trial 60 Complete [00h 00m 49s]
val_accuracy: 0.7538192272186279

Best val_accuracy So Far: 0.7606996893882751
Total elapsed time: 00h 20m 54s


In [26]:
# Start the search and get the best model:
best_model = tuner.get_best_models()[0]

# print the accuracy
print("Best accuracy:", best_model.evaluate(x_test_scaled, y_test)[1])

268/268 [==============================] - 1s 1ms/step - loss: 0.5020 - accuracy: 0.7607
Best accuracy: 0.7606996893882751


In [29]:
# Evaluate the model using the test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5020 - accuracy: 0.7607 - 636ms/epoch - 2ms/step
Loss: 0.5019620656967163, Accuracy: 0.7606996893882751


In [37]:
# Export our model to HDF5 file
# Define the filename
filename = 'C:/Users/Laura/Desktop/Boot_Camp/deep_learning_challenge_21/H5_files/AlphabetSoupCharity_Optimization.h5'


# Save the model to a HDF5 file
best_model.save(filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
